# Assignment 3 - Dependency Parser
Everything should be done ON MY code, no new code.


1. Read https://aclanthology.org/D14-1082.pdf and maybe just write one paragraph summary in your README.md in your github

2. Do something called ablation study (meaning try to delete something so we know the impact of that deleted thing - very common in NLP)
Recall that we have 18 word + 18 pos + 12 dep features
- Try to delete only the 12 dep features and check UAS
- Try to delete only the 18 pos features and check UAS

3. Do another comparison study testing the embedding
Chaky uses some embedding
Try to use (1) glove embedding (smallest), (2) nn.Embedding (train from scratch) and compare with Chaky's embedding

4. Do some testing, compare 2-3 sentences with spaCy and see whether our neural network gives the same dependency.

Criteria:
0: not done
1: ok
2: with comments/explanation like how Chaky does his tutorial

In [12]:
import sys
import numpy as np
import time
import os
import logging
from collections import Counter
from datetime import datetime
import math

from tqdm import tqdm
import pickle

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import nn, optim

# 1. Read A Fast and Accurate Dependency Parser using Neural Networks

Read https://aclanthology.org/D14-1082.pdf and maybe just write one paragraph summary in your README.md in your github [click this](https://github.com/guntsvzz/DSAI-AIT/blob/main/Course/Natural%20Language%20Understanding/Assignment/03%20-%20A%20Fast%20and%20Accurate%20Dependency%20Parser%20using%20Neural%20Networks.md)


## 1.1. Parsing function

We gonna start with a class `Parsing`, representing a parser for each sentence.  For each sentence, we need the `stack`, `buffer`, and the `dependencies`.

In [49]:
#basically, it takes the current state of the buffer, stack, dependencies
#tell us how SHIFT, LA, RA changes these three objects
class Parsing(object):
    #init stack, buffer, dep
    def __init__(self,sentence):
        self.sentence = sentence # ['The','cat','sat'] #conll format which is already in the tokenized form
        self.stack = ['ROOT']
        self.buffer = sentence[:] #in the begining, everything is inside the buffer
        self.dep    = [] #maintains a list of tuples of dep
    #parse fucntions that tells me how shift, la, ra changes these three objects
    def parse_step(self,transition): #transition could be either S, LA, RA
        if transition == 'S':
            #get the top guy in the buffer and put in stack
            head = self.buffer.pop(0)
            self.stack.append(head)
        elif transition == 'LA': #stack : [ROOT, He, has] ==> append to dep(has, he) and then He is gone from the stack [ROOT, has]
            dependent = self.stack.pop(-2) #He
            self.dep.append((self.stack[-1],dependent)) #(has, he)
        elif transition == 'RA':
            dependent = self.stack.pop(-1) #stack : [ROOT, has, control] ==> dep (has, control) 
            self.dep.append((self.stack[-1],dependent)) #(has,control)
        else:
            print(f'Bad transition : {transition}')

    #given some series of transition, it gonna for-loop the parse function
    def parse(self,transition):
        for t in transition:
            self.parse_step(t)
        return self.dep

    #check whether things are finished - no need to do anymore functions...
    def is_completed(self):
        return (len(self.buffer) == 0) and (len(self.stack) == 1)  #so buffer is empty and ROOT is the only guy in stack

### Minibatch parsing

We gonna create a minibatch loader that loads a bunch of sentences, and perform parse accordingly.  For now, we will assume a very dump model to predict the transitions.

In [47]:
def minibatch_parse(sentences, model, batch_size):
    dep = [] #all the resulting dep
    #init Parsing instacne for each sentence in the batch
    partial_parses = [Parsing(sentence) for sentence in sentences] #in tokenized form
    #Parsing(['The','cat','sat']),#Parsing(['Chaky','is','mad'])
    
    unfinished_parses = partial_parses[:]
    #while we still have sentence
    while unfinished_parses: #if there are still a Parsing object

        #take a certain batch of senetence
        minibatch = unfinished_parses[:batch_size]
        
        #create a dummy model to tell us what's the next tranisitions for each sentence
        transitions = model.predict(minibatch)

        #transitions = [S,S,....]
        #minibatch = [Parsing(sentence1),Parsing(sentence2)]
        #for transition predicted this dummy model
        for transition, partial_parse in zip(transitions,minibatch):
            #parse step
            #transition : S
            #partial_parse: Parsing(sentence)
            partial_parse.parse_step(transition)

        #remove any sentence is finish
        unfinished_parses[:] = [p for p in unfinished_parses if not p.is_completed()]
    
    dep = [parse.dep for parse in partial_parses]
    #return dep
    return dep

## 1.2. Load data

We used English Penn Treebank dataset in CoNLL format.

CoNLL is the conventional name for TSV formats in NLP (TSV - tab-separated values, i.e., CSV with <TAB> as separator).
It originates from a series of shared tasks organized at the Conferences of Natural Language Learning (hence the name)

In CoNLL formats,
- every word (token) is represented in one line
- every sentence is separated from the next by an empty line
- every column represents one annotation

There are many formats, in our case, our conll file has 10 columns, the important columns are:
- 1:  word
- 4:  pos
- 6:  head of the dependency
- 7:  type of dependency

In [30]:
def read_conll(filename):
    examples = []
    with open(filename) as f:
        i = 0
        word, pos, head, dep = [], [], [], []
        for line in f.readlines():
            i = i + 1
            wa = line.strip().split('\t') #word anotate
            #In ------> 5th guy
            #    case
            if len(wa) == 10: #if all the columns are there
                word.append(wa[1].lower())
                pos.append(wa[4])
                head.append(int(wa[6]))
                dep.append(wa[7])

            #the row is not exactly 10, it means new sentence
            elif len(word) > 0: #if there is something inside the word
                examples.append({'word':word,'pos':pos,'head':head,'dep':dep}) #in the sentence lelvel
                word, pos, head, dep = [], [], [], [] #clear word, pos, head, dep  

        if len(word) > 0: #if there is something inside the word
            examples.append({'word':word,'pos':pos,'head':head,'dep':dep}) #in the sentence lelvel
                            
    return examples
    
import time

def load_data():
    print("1. Loading data")
    start = time.time()
    train_set = read_conll('./data/train.conll')
    dev_set = read_conll('./data/dev.conll')
    test_set = read_conll('./data/test.conll')
    #make by dataset smaller because my mac cannot handle it
    train_set = train_set[:1000]
    dev_set = dev_set[:500]
    test_set = test_set[:500]
    
    print("Example 1: ", train_set[1]) #i choose one because it's short

    print("took {:.2f} seconds".format(time.time() - start))

    return train_set,dev_set,test_set

In [5]:
train_set, dev_set, test_set = load_data()
len(train_set), len(dev_set), len(test_set)

1. Loading data
Example 1:  {'word': ['ms.', 'haag', 'plays', 'elianti', '.'], 'pos': ['NNP', 'NNP', 'VBZ', 'NNP', '.'], 'head': [2, 3, 0, 3, 3], 'dep': ['compound', 'nsubj', 'root', 'dobj', 'punct']}
took 1.52 seconds


(1000, 500, 500)

## 1.3. Parser

In [32]:
P_PREFIX = '<p>:' #indicating pos tags
D_PREFIX = '<d>:' #indicating dependency tags
UNK      = '<UNK>'
NULL     = '<NULL>'
ROOT     = '<ROOT>'

class Parser(object):

    def __init__(self, dataset):
        
        #set the root dep
        self.root_dep = 'root'
                
        #get all the dep of the dataset as list, e.g., ['root', 'acl', 'nmod', 'nmod:npmod']
        all_dep = [self.root_dep] + list(set([w for ex in dataset
                                               for w in ex['dep']
                                               if w != self.root_dep]))
        
        #1. put dep into tok2id lookup table, with D_PREFIX so we know it is dependency
        #{'D_PREFIX:root': 0, 'D_PREFIX:acl': 1, 'D_PREFIX:nmod': 2, ..., 'D_PREFIX:<NULL>': 30}
        tok2id = {D_PREFIX + l: i for (i, l) in enumerate(all_dep)}
        tok2id[D_PREFIX + NULL] = self.D_NULL = len(tok2id)
        
        #we are using "unlabeled" where we do not label with the dependency
        #thus the number of dependency relation is 1
        trans = ['L', 'R', 'S']
        self.n_deprel = 1 #dependency relationship #because we ar not predicting the realtions, we are only predicting S, L, R
        
        #create a simple lookup table mapping action and id
        #e.g., tran2id: {'L': 0, 'R': 1, 'S': 2}
        #e.g., id2tran: {0: 'L', 1: 'R', 2: 'S'}
        self.n_trans = len(trans)
        self.tran2id = {t: i for (i, t) in enumerate(trans)} #use for easy coding
        self.id2tran = {i: t for (i, t) in enumerate(trans)} 

        #2. put pos tags into tok2id lookup table, with P_PREFIX so we know it is pos
        tok2id.update(build_dict([P_PREFIX + w for ex in dataset for w in ex['pos']],
                                  offset=len(tok2id)))
        tok2id[P_PREFIX + UNK]  = self.P_UNK  = len(tok2id)  #also remember the pos tags of unknown
        tok2id[P_PREFIX + NULL] = self.P_NULL = len(tok2id)
        tok2id[P_PREFIX + ROOT] = self.P_ROOT = len(tok2id)
        
        #now tok2id:  {'P_PREFIX:root': 0, 'P_PREFIX:acl': 1, ..., 'P_PREFIX:JJR': 62, 'P_PREFIX:<UNK>': 63, 'P_PREFIX:<NULL>': 64, 'P_PREFIX:<ROOT>': 65}
        
        #3. put word into tok2id lookup table
        tok2id.update(build_dict([w for ex in dataset for w in ex['word']],
                                  offset=len(tok2id)))
        tok2id[UNK]  = self.UNK = len(tok2id)
        tok2id[NULL] = self.NULL = len(tok2id)
        tok2id[ROOT] = self.ROOT = len(tok2id)
        
        #now tok2id: {'D_PREFIX:root': 0, 'D_PREFIX:acl': 1, 'D_PREFIX:nmod': 2, ..., 'memory': 340, 'mr.': 341, '<UNK>': 342, '<NULL>': 343, '<ROOT>': 344}
        
        #create id2tok
        self.tok2id = tok2id
        self.id2tok = {v: k for (k, v) in tok2id.items()}
        
        #why 18 normal features + 18 (pos) + 12 (dep)
        #18 features - top 3 words on buffer, top 3 words on stack, 
        # the first and second left most/rightmost children of the top two words on the stack
        # the leftmost of leftmost/rightmost of rightmost children of the top two words on the stack
        #18 pos - basically corresponding POS tags
        #12 dep - corresponding ARC, excluding 6 words on hte stack/buffer..
        self.n_features = 18 + 18 + 12
        self.n_tokens = len(tok2id)
        
    #utility function, in case we want to convert token to id
    #function to turn train set with words to train set with id instead using tok2id
    def numericalize(self, examples):
        numer_examples = []
        for ex in examples:
            word = [self.ROOT] + [self.tok2id[w] if w in self.tok2id
                                  else self.UNK for w in ex['word']]
            pos  = [self.P_ROOT] + [self.tok2id[P_PREFIX + w] if P_PREFIX + w in self.tok2id
                                   else self.P_UNK for w in ex['pos']]
            head = [-1] + ex['head']
            dep  = [-1] + [self.tok2id[D_PREFIX + w] if D_PREFIX + w in self.tok2id
                            else -1 for w in ex['dep']]
            numer_examples.append({'word': word, 'pos': pos,
                                 'head': head, 'dep': dep})
        return numer_examples

    #function to extract features to form a feature embedding matrix
    def extract_features(self, stack, buf, arcs, ex):
             
        #ex['word']:  [55, 32, 33, 34, 35, 30], i.e., ['root', 'ms.', 'haag', 'plays', 'elianti', '.']
        #ex['pos']:   [29, 14, 14, 16, 14, 17], i.e., ['NNP', 'NNP', 'VBZ', 'NNP', '.']
        #ex['head']:  [-1, 2, 3, 0, 3, 3]  or ['root', 'compound', 'nsubj', 'root', 'dobj', 'punct']}
        #ex['dep']:   [-1, 1, 2, 0, 6, 12] or ['compound', 'nsubj', 'root', 'dobj', 'punct']

        #stack     :  [0]
        #buffer    :  [1, 2, 3, 4, 5]
        
        if stack[0] == "ROOT":
            stack[0] = 0  #start the stack with [ROOT]

        #get leftmost children based on the dependency arcs
        def get_lc(k):
            return sorted([arc[1] for arc in arcs if arc[0] == k and arc[1] < k])

        #get right most children based on the dependency arcs
        def get_rc(k):
            return sorted([arc[1] for arc in arcs if arc[0] == k and arc[1] > k],
                          reverse=True)

        p_features = [] #pos features (2a, 2b, 2c) - 18
        d_features = [] #dep features (3b, 3c) - 12
        
        #last 3 things on the stack as features
        #if the stack is less than 3, then we simply append NULL from the left
        features = [self.NULL] * (3 - len(stack)) + [ex['word'][x] for x in stack[-3:]]
        
        # next 3 things on the buffer as features
        #if the buffer is less than 3, simply append NULL
        #the reason why NULL is appended on end because buffer is read left to right
        features += [ex['word'][x] for x in buf[:3]] + [self.NULL] * (3 - len(buf))
        
        #corresponding pos tags
        p_features = [self.P_NULL] * (3 - len(stack)) + [ex['pos'][x] for x in stack[-3:]]
        p_features += [ex['pos'][x] for x in buf[:3]] + [self.P_NULL] * (3 - len(buf))
        
        #get the leftmost and rightmost children of the top two words, thus we loop 2 times
        for i in range(2):
            if i < len(stack):
                k = stack[-i-1] #-1, -2 last two in the stack
                
                #the first and second lefmost/rightmost children of the top two words (i=1, 2) on the stack
                lc = get_lc(k)  
                rc = get_rc(k)
                
                #the leftmost of leftmost/rightmost of rightmost children of the top two words on the stack:
                llc = get_lc(lc[0]) if len(lc) > 0 else []
                rrc = get_rc(rc[0]) if len(rc) > 0 else []

                #(leftmost of first word on stack, rightmost of first word, 
                # leftmost of the second word on stack, rightmost of second, 
                # leftmost of leftmost, rightmost of rightmost
                features.append(ex['word'][lc[0]] if len(lc) > 0 else self.NULL)
                features.append(ex['word'][rc[0]] if len(rc) > 0 else self.NULL)
                features.append(ex['word'][lc[1]] if len(lc) > 1 else self.NULL)
                features.append(ex['word'][rc[1]] if len(rc) > 1 else self.NULL)
                features.append(ex['word'][llc[0]] if len(llc) > 0 else self.NULL)
                features.append(ex['word'][rrc[0]] if len(rrc) > 0 else self.NULL)

                #corresponding pos
                p_features.append(ex['pos'][lc[0]] if len(lc) > 0 else self.P_NULL)
                p_features.append(ex['pos'][rc[0]] if len(rc) > 0 else self.P_NULL)
                p_features.append(ex['pos'][lc[1]] if len(lc) > 1 else self.P_NULL)
                p_features.append(ex['pos'][rc[1]] if len(rc) > 1 else self.P_NULL)
                p_features.append(ex['pos'][llc[0]] if len(llc) > 0 else self.P_NULL)
                p_features.append(ex['pos'][rrc[0]] if len(rrc) > 0 else self.P_NULL)
            
                #corresponding dep
                d_features.append(ex['dep'][lc[0]] if len(lc) > 0 else self.D_NULL)
                d_features.append(ex['dep'][rc[0]] if len(rc) > 0 else self.D_NULL)
                d_features.append(ex['dep'][lc[1]] if len(lc) > 1 else self.D_NULL)
                d_features.append(ex['dep'][rc[1]] if len(rc) > 1 else self.D_NULL)
                d_features.append(ex['dep'][llc[0]] if len(llc) > 0 else self.D_NULL)
                d_features.append(ex['dep'][rrc[0]] if len(rrc) > 0 else self.D_NULL)
                
            else:
                #attach NULL when they don't exist
                features += [self.NULL] * 6
                p_features += [self.P_NULL] * 6
                d_features += [self.D_NULL] * 6

        features += p_features + d_features
        assert len(features) == self.n_features  #assert they are 18 + 18 + 12
        return features

    #decide whether to shift, leftarc, or rightarc, based on gold parse trees
    #this is needed to create training examples which contain samples and ground truth
    def get_oracle(self, stack, buf, ex):

        #leave if the stack is only 1, thus nothing to predict....
        if len(stack) < 2:
            return self.n_trans - 1

        #predict based on the last two words on the stack
        #stack (ROOT, he, has)
        i0 = stack[-1] #has
        i1 = stack[-2] #he

        #get the head and dependency
        h0 = ex['head'][i0]
        h1 = ex['head'][i1]
        d0 = ex['dep'][i0]
        d1 = ex['dep'][i1]

        #either shift, left arc or right arc
        #"Shift" = 2; "LA" = 0; "RA" = 1
        #if head of the second last word is the last word, then leftarc
        if (i1 > 0) and (h1 == i0):
            return 0
        #if head of the last word is the second last word, then rightarc
        #make sure nothing in the buffer has head with the last word on the stack
        #otherwise, we lose the last word.....
        elif (i1 >= 0) and (h0 == i1) and \
                (not any([x for x in buf if ex['head'][x] == i0])):
            return 1
        #otherwise shift, if something is left in buffer, otherwise, do nothing....
        else:
            return None if len(buf) == 0 else 2

    #generate training examples
    #from the training sentences and their gold parse trees 
    def create_instances(self, examples): #examples = word, pos, head, dep
        all_instances = []
        
        for i, ex in enumerate(examples):
            #Ms. Hang plays Elianti .
            #e.g., ex['word]: [344, 163, 99, 164, 165, 68]
            #here 344 stands for ROOT
            #Chaky - I chated and take a look
            n_words = len(ex['word']) - 1  #excluding the root

            #arcs = {(head, tail, dependency label)}
            stack = [0]
            buf = [i + 1 for i in range(n_words)]  #[1, 2, 3, 4, 5]
            arcs = []
            instances = []
            
            #because that's the maximum number of shift, leftarcs, rightarcs you can have
            #this will determine the sample size of each training example
            #if given five words, we will get a sample of (10, 48) where 10 comes from 5 * 2, and 48 is n_features
            #but this for loop can be break if there is nothing left....
            for i in range(n_words * 2): #during stack[0] buffer[1, 2, 3, 4, 5] #maximum times you can do either S, L, R

                #get the gold transition based on the parse trees
                #gold_t can be either shift(2), leftarc(0), or rightarc(1)
                gold_t = self.get_oracle(stack, buf, ex)
                
                #if gold_t is None, no need to extract features.....
                if gold_t is None:
                    break
                
                #make sure when the model predicts, we inform the current state of stack and buffer, so
                #the model is not allowed to make any illegal action, e.g., buffer is empty but trying to pop
                legal_labels = self.legal_labels(stack, buf)                
                assert legal_labels[gold_t] == 1
               
                #extract all the 48 features 
                features = self.extract_features(stack, buf, arcs, ex)
                instances.append((features, legal_labels, gold_t))
            
                #shift 
                if gold_t == 2:
                    stack.append(buf[0])
                    buf = buf[1:]
                #left arc 
                elif gold_t == 0:
                    arcs.append((stack[-1], stack[-2], gold_t))
                    stack = stack[:-2] + [stack[-1]]
                #right arc
                else:
                    arcs.append((stack[-2], stack[-1], gold_t - self.n_deprel))
                    stack = stack[:-1]
            else:
                all_instances += instances

        return all_instances

    #provide an one hot encoding of the labels
    def legal_labels(self, stack, buf):
        labels =  ([1] if len(stack) > 2  else [0]) * self.n_deprel #left arc but you cannot do ROOT <-----He
        labels += ([1] if len(stack) >= 2 else [0]) * self.n_deprel #right arc because ROOT ----> He
        labels += [1] if len(buf) > 0 else [0] #shift
        return labels
    
    #a simple function to check punctuation POS tags
    def punct(self, pos):
        return pos in ["''", ",", ".", ":", "``", "-LRB-", "-RRB-"]

    def parse(self, dataset, eval_batch_size=5000):
        sentences = []
        sentence_id_to_idx = {}
                
        for i, example in enumerate(dataset):
            
            #example['word']=[188, 186, 186, ..., 59]
            #n_words=37
            #sentence=[1, 2, 3, 4, 5,.., 37]
                        
            n_words = len(example['word']) - 1
            sentence = [j + 1 for j in range(n_words)]            
            sentences.append(sentence)
            
            #mapping the object unique id to the i            
            #The id is the object's memory address
            sentence_id_to_idx[id(sentence)] = i
            
        model = ModelWrapper(self, dataset, sentence_id_to_idx)
        dependencies = minibatch_parse(sentences, model, eval_batch_size)
                
        UAS = all_tokens = 0.0
        with tqdm(total=len(dataset)) as prog:
            for i, ex in enumerate(dataset):
                head = [-1] * len(ex['word'])
                for h, t, in dependencies[i]:
                    head[t] = h
                for pred_h, gold_h, gold_l, pos in \
                        zip(head[1:], ex['head'][1:], ex['dep'][1:], ex['pos'][1:]):
                        assert self.id2tok[pos].startswith(P_PREFIX)
                        pos_str = self.id2tok[pos][len(P_PREFIX):]
                        if (not self.punct(pos_str)):
                            UAS += 1 if pred_h == gold_h else 0
                            all_tokens += 1
                prog.update(i + 1)
        UAS /= all_tokens
        return UAS, dependencies

In [33]:
class ModelWrapper(object):
    def __init__(self, parser, dataset, sentence_id_to_idx):
        self.parser = parser
        self.dataset = dataset
        self.sentence_id_to_idx = sentence_id_to_idx

    def predict(self, partial_parses):
        mb_x = [self.parser.extract_features(p.stack, p.buffer, p.dep,
                                             self.dataset[self.sentence_id_to_idx[id(p.sentence)]])
                for p in partial_parses]
        mb_x = np.array(mb_x).astype('int32')
        mb_x = torch.from_numpy(mb_x).long()
        mb_l = [self.parser.legal_labels(p.stack, p.buffer) for p in partial_parses]

        pred = self.parser.model(mb_x)
        pred = pred.detach().numpy()
        
        #we need to multiply 10000 with legal labels, to force the model not to make any impossible prediction
        #other, when we parse sequentially, sometimes there is nothing in the buffer or stack, thus error....        
        pred = np.argmax(pred + 10000 * np.array(mb_l).astype('float32'), 1)
        pred = ["S" if p == 2 else ("LA" if p == 0 else "RA") for p in pred]
        
        return pred

In [34]:
#a simple function to create ids.....
def build_dict(keys, offset=0):
    #keys = ['P_PREFIX:IN', 'P_PREFIX:DT', 'P_PREFIX:NNP', 'P_PREFIX:CD', so on...]
    #offset is needed because this tok2id has something already inside....
    count = Counter()
    for key in keys:
        count[key] += 1
    
    #most_common =X [('P_PREFIX:NN', 70), ('P_PREFIX:IN', 57), ... , ('P_PREFIX:JJR', 1)]
    #we use most_common in case we only want some maximum pos tags....
    mc = count.most_common()
    
    #{'P_PREFIX:NN': 31, 'P_PREFIX:IN': 32, .., 'P_PREFIX:JJR': 62} 
    return {w[0]: index + offset for (index, w) in enumerate(mc)}

In [9]:
print('2. Building parser....')
start = time.time()
parser = Parser(train_set)
print("took {:.2f} seconds".format(time.time()-start))

2. Building parser....
took 0.02 seconds


In [10]:
#before numericalize
print('Word: ',train_set[1]['word'])
print('Pos: ',train_set[1]['pos'])
print('Head: ',train_set[1]['head'])
print('Dep: ',train_set[1]['dep'])

Word:  ['ms.', 'haag', 'plays', 'elianti', '.']
Pos:  ['NNP', 'NNP', 'VBZ', 'NNP', '.']
Head:  [2, 3, 0, 3, 3]
Dep:  ['compound', 'nsubj', 'root', 'dobj', 'punct']


In [11]:
train_set = parser.numericalize(train_set)
dev_set = parser.numericalize(dev_set)
test_set = parser.numericalize(test_set)

In [12]:
#after numericalize (rootis added in front)
print('Word: ',train_set[1]['word'])
print('Pos: ',train_set[1]['pos'])
print('Head: ',train_set[1]['head'])
print('Dep: ',train_set[1]['dep'])

Word:  [5156, 304, 1364, 1002, 2144, 87]
Pos:  [84, 42, 42, 55, 42, 46]
Head:  [-1, 2, 3, 0, 3, 3]
Dep:  [-1, 27, 28, 0, 21, 12]


## 1.4. Word Embedding

Word embedding length of 50.  In the paper, they applied a custom 50-embedding to all the words, pos, and dependencies.  For pos and dependencies, they claimed that there are some similarities that can be learned as well.

In [13]:
print("4. Loading pretrained embeddings...",)
# config = Config()
start = time.time()
word_vectors = {}
for line in open('./data/en-cw.txt').readlines():
    we = line.strip().split() #we = word embeddings - first column: word;  the rest is embedding
    word_vectors[we[0]] = [float(x) for x in we[1:]] #{word: [list of 50 numbers], nextword: [another list], so on...}
    
#create an empty embedding matrix holding the embedding lookup table (vocab size, embed dim)
#we use random.normal instead of zeros, to keep the embedding matrix arbitrary in case word vectors don't exist....
embeddings_matrix = np.asarray(np.random.normal(0, 0.9, (parser.n_tokens, 50)), dtype='float32')

for token in parser.tok2id:
        i = parser.tok2id[token]
        if token in word_vectors:
            embeddings_matrix[i] = word_vectors[token]
        elif token.lower() in word_vectors:
            embeddings_matrix[i] = word_vectors[token.lower()]
print("Embedding matrix shape (vocab, emb size): ", embeddings_matrix.shape)
print("took {:.2f} seconds".format(time.time() - start))

4. Loading pretrained embeddings...
Embedding matrix shape (vocab, emb size):  (5157, 50)
took 2.15 seconds


## 1.5. Preprocessing

In [14]:
print("5. Preprocessing training data...",)
start = time.time()
train_examples = parser.create_instances(train_set)
print("took {:.2f} seconds".format(time.time() - start))

5. Preprocessing training data...
took 1.21 seconds


## 1.6. Minibatch loader

In [45]:
def get_minibatches(data, minibatch_size, shuffle=True):
    data_size = len(data[0])
    indices = np.arange(data_size)
    if shuffle:
        np.random.shuffle(indices)
    for minibatch_start in np.arange(0, data_size, minibatch_size):
        minibatch_indices = indices[minibatch_start:minibatch_start + minibatch_size]
        yield [_minibatch(d, minibatch_indices) for d in data]

def _minibatch(data, minibatch_idx):
    return data[minibatch_idx] if type(data) is np.ndarray else [data[i] for i in minibatch_idx]

def minibatches(data, batch_size):
    x = np.array([d[0] for d in data])
    y = np.array([d[2] for d in data])
    one_hot = np.zeros((y.size, 3))
    one_hot[np.arange(y.size), y] = 1
    return get_minibatches([x, one_hot], batch_size)

## 1.7. Neural Network

In [41]:
class ParserModel(nn.Module):
    def __init__(self, embeddings_matrix, hidden_size = 200, embed_size = 50,n_features = 48):
        super(ParserModel, self).__init__()
        self.transition_size = 3
        self.n_features = n_features
        self.embed_size = embeddings_matrix.shape[1]
        self.pretrained_embeddings = nn.Embedding(embeddings_matrix.shape[0], self.embed_size)
        self.pretrained_embeddings.weight = nn.Parameter(torch.tensor(embeddings_matrix))
        self.hidden1 = nn.Linear(self.n_features * embed_size, hidden_size) 
        self.hidden2 = nn.Linear(hidden_size, self.transition_size)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)

    def embedding_lookup(self, x):
        batch_size = x.size()[0]
        x = self.pretrained_embeddings(x)        
        x = x.reshape(-1, self.n_features * self.embed_size) # x = (1024, 48 * 50)
        return x

    def forward(self,x):
        #x: (batch_size,48)
        #goes to the embeddding layer ==> (batch_sizem, 40 * emb_size)
        input_embed = self.embedding_lookup(x)
        #goes through the linear layer ==> (batch_sizem, 40 *hid_size)
        h1 = self.dropout(self.relu(self.hidden1(input_embed)))
        #do relu then dropout
        #compute the logits (basically a linear layer that converts to (batch_size,transition_size) ==> (batch_size,3))
        logits = self.hidden2(h1)
        return logits

In [42]:
#just a class to get the average.....
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [43]:
def train(parser, train_data, dev_data, output_path, batch_size=1024, n_epochs=10, lr=0.0005):
    
    best_dev_UAS = 0
    
    optimizer = optim.Adam(parser.model.parameters(), lr=0.001)
    loss_func = nn.CrossEntropyLoss()

    for epoch in range(n_epochs):
        print("Epoch {:} out of {:}".format(epoch + 1, n_epochs))
        dev_UAS = train_for_epoch(
            parser, train_data, dev_data, optimizer, loss_func, batch_size)
        if dev_UAS > best_dev_UAS:
            best_dev_UAS = dev_UAS
            print("New best dev UAS! Saving model.")
            torch.save(parser.model.state_dict(), output_path)
        print("")


def train_for_epoch(parser, train_data, dev_data, optimizer, loss_func, batch_size):
    
    parser.model.train()  # Places model in "train" mode, i.e. apply dropout layer
    n_minibatches = math.ceil(len(train_data) / batch_size)
    loss_meter = AverageMeter()

    with tqdm(total=(n_minibatches)) as prog:
        for i, (train_x, train_y) in enumerate(minibatches(train_data, batch_size)):
            
            #train_x:  batch_size, n_features
            #train_y:  batch_size, target(=3)
            
            optimizer.zero_grad() 
            loss = 0.
            train_x = torch.from_numpy(train_x).long()  #long() for int so embedding works....
            train_y = torch.from_numpy(train_y.nonzero()[1]).long()  #get the index with 1 because torch expects label to be single integer

            # Forward pass: compute predicted logits.
            logits = parser.model(train_x)
            # Compute loss
            loss = loss_func(logits, train_y)
            # Compute gradients of the loss w.r.t model parameters.
            loss.backward()
            # Take step with optimizer.
            optimizer.step()

            prog.update(1)
            loss_meter.update(loss.item())

    print("Average Train Loss: {}".format(loss_meter.avg))
    print("Evaluating on dev set",)
    parser.model.eval()  # Places model in "eval" mode, i.e. don't apply dropout layer
        
    dev_UAS, _ = parser.parse(dev_data)
    print("- dev UAS: {:.2f}".format(dev_UAS * 100.0))
    return dev_UAS

## 1.8. Training

In [19]:
#create directory if it does not exist for saving the weights...
output_dir = "output/{:%Y%m%d_%H%M%S}/".format(datetime.now())
output_path = output_dir + "model.weights"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    
print(80 * "=")
print("TRAINING")
print(80 * "=")
    
model = ParserModel(embeddings_matrix)
parser.model = model

start = time.time()
train(parser, train_examples, dev_set, output_path,
      batch_size=1024, n_epochs=10, lr=0.0005)

TRAINING
Epoch 1 out of 10


100%|██████████| 48/48 [00:03<00:00, 12.19it/s]


Average Train Loss: 0.5608917164305846
Evaluating on dev set


125250it [00:00, 10377838.76it/s]      


- dev UAS: 57.02
New best dev UAS! Saving model.

Epoch 2 out of 10


...

- dev UAS: 74.11
New best dev UAS! Saving model.

Epoch 10 out of 10


100%|██████████| 48/48 [00:02<00:00, 17.26it/s]


Average Train Loss: 0.13013719130928317
Evaluating on dev set


125250it [00:00, 12469713.88it/s]      

- dev UAS: 73.55



## 1.9. Testing

In [22]:
print(80 * "=")
print("TESTING")
print(80 * "=")

print("Restoring the best model weights found on the dev set")
parser.model.load_state_dict(torch.load(output_path))
print("Final evaluation on test set",)
parser.model.eval()
UAS, dependencies = parser.parse(test_set)
print("- test UAS: {:.2f}".format(UAS * 100.0))
print("Done!")

TESTING
Restoring the best model weights found on the dev set
Final evaluation on test set


125250it [00:00, 13886037.64it/s]      

- test UAS: 75.94
Done!


# 2. Ablation Study 
Do something called ablation study (meaning try to delete something so we know the impact of that deleted thing - very common in NLP)
Recall that we have 18 word + 18 pos + 12 dep features

In [5]:
P_PREFIX = '<p>:' #indicating pos tags
D_PREFIX = '<d>:' #indicating dependency tags
UNK      = '<UNK>'
NULL     = '<NULL>'
ROOT     = '<ROOT>'

class ablationStudyParser(object):

    def __init__(self, dataset, pos_f = True, dep_f = True):
        
        #set pos and dep feature
        self.pos_f = pos_f
        self.dep_f = dep_f
        tok2id = dict() 
        
        #### Dependency Features : If it is False, dependency relationship will prevent
        if self.dep_f : 
            #set the root dep
            self.root_dep = 'root'
                
            #get all the dep of the dataset as list, e.g., ['root', 'acl', 'nmod', 'nmod:npmod']
            all_dep = [self.root_dep] + list(set([w for ex in dataset
                                               for w in ex['dep']
                                               if w != self.root_dep]))
        
            #1. put dep into tok2id lookup table, with D_PREFIX so we know it is dependency
            #{'D_PREFIX:root': 0, 'D_PREFIX:acl': 1, 'D_PREFIX:nmod': 2, ..., 'D_PREFIX:<NULL>': 30}
            tok2id = {D_PREFIX + l: i for (i, l) in enumerate(all_dep)}
            tok2id[D_PREFIX + NULL] = self.D_NULL = len(tok2id)
        
        #we are using "unlabeled" where we do not label with the dependency
        #thus the number of dependency relation is 1
        trans = ['L', 'R', 'S']
        self.n_deprel = 1 #dependency relationship #because we ar not predicting the realtions, we are only predicting S, L, R
        
        #create a simple lookup table mapping action and id
        #e.g., tran2id: {'L': 0, 'R': 1, 'S': 2}
        #e.g., id2tran: {0: 'L', 1: 'R', 2: 'S'}
        self.n_trans = len(trans)
        self.tran2id = {t: i for (i, t) in enumerate(trans)} #use for easy coding
        self.id2tran = {i: t for (i, t) in enumerate(trans)} 
        
        #### POS tag features : If it is False, POS tag will not be added as feature
        if self.pos_f : 
            #2. put pos tags into tok2id lookup table, with P_PREFIX so we know it is pos
            tok2id.update(build_dict([P_PREFIX + w for ex in dataset for w in ex['pos']],
                                    offset=len(tok2id)))
            tok2id[P_PREFIX + UNK]  = self.P_UNK  = len(tok2id)  #also remember the pos tags of unknown
            tok2id[P_PREFIX + NULL] = self.P_NULL = len(tok2id)
            tok2id[P_PREFIX + ROOT] = self.P_ROOT = len(tok2id)
        
        #now tok2id:  {'P_PREFIX:root': 0, 'P_PREFIX:acl': 1, ..., 'P_PREFIX:JJR': 62, 'P_PREFIX:<UNK>': 63, 'P_PREFIX:<NULL>': 64, 'P_PREFIX:<ROOT>': 65}
        
        #### Word Features 
        #3. put word into tok2id lookup table
        tok2id.update(build_dict([w for ex in dataset for w in ex['word']],
                                  offset=len(tok2id)))
        tok2id[UNK]  = self.UNK = len(tok2id)
        tok2id[NULL] = self.NULL = len(tok2id)
        tok2id[ROOT] = self.ROOT = len(tok2id)
        
        #now tok2id: {'D_PREFIX:root': 0, 'D_PREFIX:acl': 1, 'D_PREFIX:nmod': 2, ..., 'memory': 340, 'mr.': 341, '<UNK>': 342, '<NULL>': 343, '<ROOT>': 344}
        
        #create id2tok
        self.tok2id = tok2id
        self.id2tok = {v: k for (k, v) in tok2id.items()}
        
        #why 18 normal features + 18 (pos) + 12 (dep)
        #18 features - top 3 words on buffer, top 3 words on stack, 
        # the first and second left most/rightmost children of the top two words on the stack
        # the leftmost of leftmost/rightmost of rightmost children of the top two words on the stack
        #18 pos - basically corresponding POS tags
        #12 dep - corresponding ARC, excluding 6 words on hte stack/buffer..

        if self.dep_f and self.pos_f:
            self.n_features = 18 + 18 + 12 # 18 normal features + 18 (pos) + 12 (dep)
        elif self.pos_f :
            self.n_features = 18 + 18 # 18 normal features + 18 (pos) 
        elif self.dep_f :
            self.n_features = 18 + 12 # 18 normal features + 12 (dep)
        else:
            self.n_features = 18 # 18 normal features

        self.n_tokens = len(tok2id)
        
    #utility function, in case we want to convert token to id
    #function to turn train set with words to train set with id instead using tok2id
    def numericalize(self, examples):
        numer_examples = []
        for ex in examples:
            word = [self.ROOT] + [self.tok2id[w] if w in self.tok2id
                                  else self.UNK for w in ex['word']]
            if self.pos_f:
                pos  = [self.P_ROOT] + [self.tok2id[P_PREFIX + w] if P_PREFIX + w in self.tok2id
                                   else self.P_UNK for w in ex['pos']]
            head = [-1] + ex['head']
            if self.dep_f:
                dep  = [-1] + [self.tok2id[D_PREFIX + w] if D_PREFIX + w in self.tok2id
                            else -1 for w in ex['dep']]
            
            if self.dep_f and self.pos_f:
                numer_examples.append({'word': word, 'pos': pos,
                                 'head': head, 'dep': dep})
            elif self.pos_f :
                numer_examples.append({'word': word, 'pos': pos,
                                 'head': head})
            elif self.dep_f :
                numer_examples.append({'word': word,
                                 'head': head, 'dep': dep})
            else:
                numer_examples.append({'word': word, 
                                 'head': head})
        return numer_examples

    #function to extract features to form a feature embedding matrix
    def extract_features(self, stack, buf, arcs, ex):
             
        #ex['word']:  [55, 32, 33, 34, 35, 30], i.e., ['root', 'ms.', 'haag', 'plays', 'elianti', '.']
        #ex['pos']:   [29, 14, 14, 16, 14, 17], i.e., ['NNP', 'NNP', 'VBZ', 'NNP', '.']
        #ex['head']:  [-1, 2, 3, 0, 3, 3]  or ['root', 'compound', 'nsubj', 'root', 'dobj', 'punct']}
        #ex['dep']:   [-1, 1, 2, 0, 6, 12] or ['compound', 'nsubj', 'root', 'dobj', 'punct']

        #stack     :  [0]
        #buffer    :  [1, 2, 3, 4, 5]
        
        if stack[0] == "ROOT":
            stack[0] = 0  #start the stack with [ROOT]

        #get leftmost children based on the dependency arcs
        def get_lc(k):
            return sorted([arc[1] for arc in arcs if arc[0] == k and arc[1] < k])

        #get right most children based on the dependency arcs
        def get_rc(k):
            return sorted([arc[1] for arc in arcs if arc[0] == k and arc[1] > k],
                          reverse=True)

        p_features = [] #pos features (2a, 2b, 2c) - 18
        d_features = [] #dep features (3b, 3c) - 12
        
        #last 3 things on the stack as features
        #if the stack is less than 3, then we simply append NULL from the left
        features = [self.NULL] * (3 - len(stack)) + [ex['word'][x] for x in stack[-3:]]
        
        # next 3 things on the buffer as features
        #if the buffer is less than 3, simply append NULL
        #the reason why NULL is appended on end because buffer is read left to right
        features += [ex['word'][x] for x in buf[:3]] + [self.NULL] * (3 - len(buf))
        
        if self.pos_f :
            #corresponding pos tags
            p_features = [self.P_NULL] * (3 - len(stack)) + [ex['pos'][x] for x in stack[-3:]]
            p_features += [ex['pos'][x] for x in buf[:3]] + [self.P_NULL] * (3 - len(buf))
        
        #get the leftmost and rightmost children of the top two words, thus we loop 2 times
        for i in range(2):
            if i < len(stack):
                k = stack[-i-1] #-1, -2 last two in the stack
                
                #the first and second lefmost/rightmost children of the top two words (i=1, 2) on the stack
                lc = get_lc(k)  
                rc = get_rc(k)
                
                #the leftmost of leftmost/rightmost of rightmost children of the top two words on the stack:
                llc = get_lc(lc[0]) if len(lc) > 0 else []
                rrc = get_rc(rc[0]) if len(rc) > 0 else []

                #(leftmost of first word on stack, rightmost of first word, 
                # leftmost of the second word on stack, rightmost of second, 
                # leftmost of leftmost, rightmost of rightmost
                features.append(ex['word'][lc[0]] if len(lc) > 0 else self.NULL)
                features.append(ex['word'][rc[0]] if len(rc) > 0 else self.NULL)
                features.append(ex['word'][lc[1]] if len(lc) > 1 else self.NULL)
                features.append(ex['word'][rc[1]] if len(rc) > 1 else self.NULL)
                features.append(ex['word'][llc[0]] if len(llc) > 0 else self.NULL)
                features.append(ex['word'][rrc[0]] if len(rrc) > 0 else self.NULL)

                if self.pos_f :
                    #corresponding pos
                    p_features.append(ex['pos'][lc[0]] if len(lc) > 0 else self.P_NULL)
                    p_features.append(ex['pos'][rc[0]] if len(rc) > 0 else self.P_NULL)
                    p_features.append(ex['pos'][lc[1]] if len(lc) > 1 else self.P_NULL)
                    p_features.append(ex['pos'][rc[1]] if len(rc) > 1 else self.P_NULL)
                    p_features.append(ex['pos'][llc[0]] if len(llc) > 0 else self.P_NULL)
                    p_features.append(ex['pos'][rrc[0]] if len(rrc) > 0 else self.P_NULL)
                if self.dep_f :
                    #corresponding dep
                    d_features.append(ex['dep'][lc[0]] if len(lc) > 0 else self.D_NULL)
                    d_features.append(ex['dep'][rc[0]] if len(rc) > 0 else self.D_NULL)
                    d_features.append(ex['dep'][lc[1]] if len(lc) > 1 else self.D_NULL)
                    d_features.append(ex['dep'][rc[1]] if len(rc) > 1 else self.D_NULL)
                    d_features.append(ex['dep'][llc[0]] if len(llc) > 0 else self.D_NULL)
                    d_features.append(ex['dep'][rrc[0]] if len(rrc) > 0 else self.D_NULL)
                
            else:
                #attach NULL when they don't exist
                features += [self.NULL] * 6
                if self.pos_f :
                    p_features += [self.P_NULL] * 6
                if self.dep_f :
                    d_features += [self.D_NULL] * 6

        if self.dep_f and self.pos_f:
            features += p_features + d_features
        elif self.pos_f :
            features += p_features
        elif self.dep_f :
            features += d_features

        assert len(features) == self.n_features  #assert they are 18 + 18 + 12 or 18 + 18 or 18 + 18 + 12
        return features

    #decide whether to shift, leftarc, or rightarc, based on gold parse trees
    #this is needed to create training examples which contain samples and ground truth
    def get_oracle(self, stack, buf, ex):

        #leave if the stack is only 1, thus nothing to predict....
        if len(stack) < 2:
            return self.n_trans - 1

        #predict based on the last two words on the stack
        #stack (ROOT, he, has)
        i0 = stack[-1] #has
        i1 = stack[-2] #he

        #get the head and dependency
        h0 = ex['head'][i0]
        h1 = ex['head'][i1]

        if self.dep_f :
            d0 = ex['dep'][i0]
            d1 = ex['dep'][i1]

        #either shift, left arc or right arc
        #"Shift" = 2; "LA" = 0; "RA" = 1
        #if head of the second last word is the last word, then leftarc
        if (i1 > 0) and (h1 == i0):
            return 0
        #if head of the last word is the second last word, then rightarc
        #make sure nothing in the buffer has head with the last word on the stack
        #otherwise, we lose the last word.....
        elif (i1 >= 0) and (h0 == i1) and \
                (not any([x for x in buf if ex['head'][x] == i0])):
            return 1
        #otherwise shift, if something is left in buffer, otherwise, do nothing....
        else:
            return None if len(buf) == 0 else 2

    #generate training examples
    #from the training sentences and their gold parse trees 
    def create_instances(self, examples): #examples = word, pos, head, dep
        all_instances = []
        
        for i, ex in enumerate(examples):
            #Ms. Hang plays Elianti .
            #e.g., ex['word]: [344, 163, 99, 164, 165, 68]
            #here 344 stands for ROOT
            #Chaky - I chated and take a look
            n_words = len(ex['word']) - 1  #excluding the root

            #arcs = {(head, tail, dependency label)}
            stack = [0]
            buf = [i + 1 for i in range(n_words)]  #[1, 2, 3, 4, 5]
            arcs = []
            instances = []
            
            #because that's the maximum number of shift, leftarcs, rightarcs you can have
            #this will determine the sample size of each training example
            #if given five words, we will get a sample of (10, 48) where 10 comes from 5 * 2, and 48 is n_features
            #but this for loop can be break if there is nothing left....
            for i in range(n_words * 2): #during stack[0] buffer[1, 2, 3, 4, 5] #maximum times you can do either S, L, R

                #get the gold transition based on the parse trees
                #gold_t can be either shift(2), leftarc(0), or rightarc(1)
                gold_t = self.get_oracle(stack, buf, ex)
                
                #if gold_t is None, no need to extract features.....
                if gold_t is None:
                    break
                
                #make sure when the model predicts, we inform the current state of stack and buffer, so
                #the model is not allowed to make any illegal action, e.g., buffer is empty but trying to pop
                legal_labels = self.legal_labels(stack, buf)                
                assert legal_labels[gold_t] == 1
               
                #extract all the 48 features 
                features = self.extract_features(stack, buf, arcs, ex)
                instances.append((features, legal_labels, gold_t))
            
                #shift 
                if gold_t == 2:
                    stack.append(buf[0])
                    buf = buf[1:]
                #left arc 
                elif gold_t == 0:
                    arcs.append((stack[-1], stack[-2], gold_t))
                    stack = stack[:-2] + [stack[-1]]
                #right arc
                else:
                    arcs.append((stack[-2], stack[-1], gold_t - self.n_deprel))
                    stack = stack[:-1]
            else:
                all_instances += instances

        if not self.dep_f:
            all_instances = [[instance[0], instance[2]] for instance in all_instances] #return only 'word' and 'head'

        return all_instances

    #provide an one hot encoding of the labels
    def legal_labels(self, stack, buf):
        labels =  ([1] if len(stack) > 2  else [0]) * self.n_deprel #left arc but you cannot do ROOT <-----He
        labels += ([1] if len(stack) >= 2 else [0]) * self.n_deprel #right arc because ROOT ----> He
        labels += [1] if len(buf) > 0 else [0] #shift
        return labels
    
    #a simple function to check punctuation POS tags
    def punct(self, pos):
        return pos in ["''", ",", ".", ":", "``", "-LRB-", "-RRB-"]

    def parse(self, dataset, eval_batch_size=5000):
        sentences = []
        sentence_id_to_idx = {}
                
        for i, example in enumerate(dataset):
            
            #example['word']=[188, 186, 186, ..., 59]
            #n_words=37
            #sentence=[1, 2, 3, 4, 5,.., 37]
                        
            n_words = len(example['word']) - 1
            sentence = [j + 1 for j in range(n_words)]            
            sentences.append(sentence)
            
            #mapping the object unique id to the i            
            #The id is the object's memory address
            sentence_id_to_idx[id(sentence)] = i
            
        model = ModelWrapper(self, dataset, sentence_id_to_idx)
        dependencies = minibatch_parse(sentences, model, eval_batch_size)
                
        UAS = all_tokens = 0.0
        with tqdm(total=len(dataset)) as prog:
            for i, ex in enumerate(dataset):
                head = [-1] * len(ex['word'])
                for h, t, in dependencies[i]:
                    head[t] = h
                if self.dep_f and self.pos_f:
                    for pred_h, gold_h, gold_l, pos in \
                            zip(head[1:], ex['head'][1:], ex['dep'][1:], ex['pos'][1:]):
                            assert self.id2tok[pos].startswith(P_PREFIX)
                            pos_str = self.id2tok[pos][len(P_PREFIX):]
                            if (not self.punct(pos_str)):
                                UAS += 1 if pred_h == gold_h else 0
                                all_tokens += 1
                elif self.pos_f :
                    for pred_h, gold_h, pos in \
                            zip(head[1:], ex['head'][1:], ex['pos'][1:]):
                            assert self.id2tok[pos].startswith(P_PREFIX)
                            pos_str = self.id2tok[pos][len(P_PREFIX):]
                            if (not self.punct(pos_str)):
                                UAS += 1 if pred_h == gold_h else 0
                                all_tokens += 1
                elif self.dep_f :
                    for pred_h, gold_h, gold_l in \
                            zip(head[1:], ex['head'][1:], ex['dep'][1:]):
                            UAS += 1 if pred_h == gold_h else 0
                            all_tokens += 1
                else:
                    for pred_h, gold_h in \
                            zip(head[1:], ex['head'][1:]):
                            UAS += 1 if pred_h == gold_h else 0
                            all_tokens += 1

                prog.update(i + 1)
        UAS /= all_tokens
        return UAS, dependencies

class ModelWrapper(object):
    def __init__(self, parser, dataset, sentence_id_to_idx):
        self.parser = parser
        self.dataset = dataset
        self.sentence_id_to_idx = sentence_id_to_idx

    def predict(self, partial_parses):
        mb_x = [self.parser.extract_features(p.stack, p.buffer, p.dep,
                                             self.dataset[self.sentence_id_to_idx[id(p.sentence)]])
                for p in partial_parses]
        mb_x = np.array(mb_x).astype('int32')
        mb_x = torch.from_numpy(mb_x).long()
        mb_l = [self.parser.legal_labels(p.stack, p.buffer) for p in partial_parses]

        pred = self.parser.model(mb_x)
        pred = pred.detach().numpy()
        
        #we need to multiply 10000 with legal labels, to force the model not to make any impossible prediction
        #other, when we parse sequentially, sometimes there is nothing in the buffer or stack, thus error....        
        pred = np.argmax(pred + 10000 * np.array(mb_l).astype('float32'), 1)
        pred = ["S" if p == 2 else ("LA" if p == 0 else "RA") for p in pred]
        
        return pred

In [6]:
def get_minibatches(data, minibatch_size, shuffle=True):
    data_size = len(data[0])
    indices = np.arange(data_size)
    if shuffle:
        np.random.shuffle(indices)
    for minibatch_start in np.arange(0, data_size, minibatch_size):
        minibatch_indices = indices[minibatch_start:minibatch_start + minibatch_size]
        yield [_minibatch(d, minibatch_indices) for d in data]

def _minibatch(data, minibatch_idx):
    return data[minibatch_idx] if type(data) is np.ndarray else [data[i] for i in minibatch_idx]
           
def minibatches(data, batch_size, dep_f = True):
    if dep_f :
        x = np.array([d[0] for d in data])
        y = np.array([d[2] for d in data])
        one_hot = np.zeros((y.size, 3))
        one_hot[np.arange(y.size), y] = 1
    else :
        x = np.array([d[0] for d in data])
        y = np.array([d[1] for d in data])
        one_hot = np.zeros((y.size, 3))
        one_hot[np.arange(y.size), y] = 1

    return get_minibatches([x, one_hot], batch_size)
    
def train(parser, train_data, dev_data, output_path, batch_size=1024, n_epochs=10, lr=0.0005, dep_f = True):
    
    best_dev_UAS = 0
    
    optimizer = optim.Adam(parser.model.parameters(), lr=0.001)
    loss_func = nn.CrossEntropyLoss()

    for epoch in range(n_epochs):
        print("Epoch {:} out of {:}".format(epoch + 1, n_epochs))
        dev_UAS = train_for_epoch(
            parser, train_data, dev_data, optimizer, loss_func, batch_size, dep_f)
        if dev_UAS > best_dev_UAS:
            best_dev_UAS = dev_UAS
            print("New best dev UAS! Saving model.")
            torch.save(parser.model.state_dict(), output_path)
        print("")


def train_for_epoch(parser, train_data, dev_data, optimizer, loss_func, batch_size, dep_f):
    
    parser.model.train()  # Places model in "train" mode, i.e. apply dropout layer
    n_minibatches = math.ceil(len(train_data) / batch_size)
    loss_meter = AverageMeter()

    with tqdm(total=(n_minibatches)) as prog:
        for i, (train_x, train_y) in enumerate(minibatches(train_data, batch_size, dep_f)):
            
            #train_x:  batch_size, n_features
            #train_y:  batch_size, target(=3)
            
            optimizer.zero_grad() 
            loss = 0.
            train_x = torch.from_numpy(train_x).long()  #long() for int so embedding works....
            train_y = torch.from_numpy(train_y.nonzero()[1]).long()  #get the index with 1 because torch expects label to be single integer

            # Forward pass: compute predicted logits.
            logits = parser.model(train_x)
            # Compute loss
            loss = loss_func(logits, train_y)
            # Compute gradients of the loss w.r.t model parameters.
            loss.backward()
            # Take step with optimizer.
            optimizer.step()

            prog.update(1)
            loss_meter.update(loss.item())

    print("Average Train Loss: {}".format(loss_meter.avg))
    print("Evaluating on dev set",)
    parser.model.eval()  # Places model in "eval" mode, i.e. don't apply dropout layer
    
    dev_UAS, _ = parser.parse(dev_data)
    print("- dev UAS: {:.2f}".format(dev_UAS * 100.0))
    return dev_UAS

- Try to delete only the 12 dep features and check UAS

In [265]:
train_set, dev_set, test_set = load_data()
# len(train_set), len(dev_set), len(test_set)
print('2. Building parser....')
start = time.time()
parser = ablationStudyParser(train_set,dep_f=False)
print("took {:.2f} seconds".format(time.time()-start))

1. Loading data
Example 1:  {'word': ['ms.', 'haag', 'plays', 'elianti', '.'], 'pos': ['NNP', 'NNP', 'VBZ', 'NNP', '.'], 'head': [2, 3, 0, 3, 3], 'dep': ['compound', 'nsubj', 'root', 'dobj', 'punct']}
took 1.78 seconds
2. Building parser....
took 0.02 seconds


In [266]:
#before numericalize
print('Word: ',train_set[1]['word'])
print('Pos: ',train_set[1]['pos'])
print('Head: ',train_set[1]['head'])
# print('Dep: ',train_set[1]['dep'])
train_set = parser.numericalize(train_set)
dev_set = parser.numericalize(dev_set)
test_set = parser.numericalize(test_set)
#after numericalize (rootis added in front)
print('Word: ',train_set[1]['word'])
print('Pos: ',train_set[1]['pos'])
print('Head: ',train_set[1]['head'])
# print('Dep: ',train_set[1]['dep'])

Word:  ['ms.', 'haag', 'plays', 'elianti', '.']
Pos:  ['NNP', 'NNP', 'VBZ', 'NNP', '.']
Head:  [2, 3, 0, 3, 3]
Word:  [5117, 265, 1325, 963, 2105, 48]
Pos:  [45, 3, 3, 16, 3, 7]
Head:  [-1, 2, 3, 0, 3, 3]


In [267]:
print("4. Loading pretrained embeddings...",)
# config = Config()
start = time.time()
word_vectors = {}
for line in open('./data/en-cw.txt').readlines():
    we = line.strip().split() #we = word embeddings - first column: word;  the rest is embedding
    word_vectors[we[0]] = [float(x) for x in we[1:]] #{word: [list of 50 numbers], nextword: [another list], so on...}
    
#create an empty embedding matrix holding the embedding lookup table (vocab size, embed dim)
#we use random.normal instead of zeros, to keep the embedding matrix arbitrary in case word vectors don't exist....
embeddings_matrix = np.asarray(np.random.normal(0, 0.9, (parser.n_tokens, 50)), dtype='float32')

for token in parser.tok2id:
        i = parser.tok2id[token]
        if token in word_vectors:
            embeddings_matrix[i] = word_vectors[token]
        elif token.lower() in word_vectors:
            embeddings_matrix[i] = word_vectors[token.lower()]
print("Embedding matrix shape (vocab, emb size): ", embeddings_matrix.shape)
print("took {:.2f} seconds".format(time.time() - start))

print("5. Preprocessing training data...",)
start = time.time()
train_examples_without12dep = parser.create_instances(train_set)
print("took {:.2f} seconds".format(time.time() - start))

4. Loading pretrained embeddings...
Embedding matrix shape (vocab, emb size):  (5118, 50)
took 1.99 seconds
5. Preprocessing training data...
took 1.13 seconds


In [268]:
#create directory if it does not exist for saving the weights...
output_path = "./output/modelwithout12dep.weights"

print(80 * "=")
print("TRAINING")
print(80 * "=")
    
model = ParserModel(embeddings_matrix, n_features = 36)
parser.model = model

start = time.time()
train(parser, train_examples_without12dep, dev_set, output_path,
      batch_size=1024, n_epochs=10, lr=0.0005, dep_f = False)

TRAINING
Epoch 1 out of 10


100%|██████████| 48/48 [00:02<00:00, 23.57it/s]


Average Train Loss: 0.5434817324082056
Evaluating on dev set


125250it [00:00, 11927269.29it/s]      


- dev UAS: 60.48
New best dev UAS! Saving model.

Epoch 2 out of 10


...

- dev UAS: 75.53
New best dev UAS! Saving model.

Epoch 10 out of 10


100%|██████████| 48/48 [00:02<00:00, 23.53it/s]


Average Train Loss: 0.11588267919917901
Evaluating on dev set


125250it [00:00, 10824540.01it/s]      

- dev UAS: 74.78



In [269]:
print(80 * "=")
print("TESTING")
print(80 * "=")

print("Restoring the best model weights found on the dev set")
parser.model.load_state_dict(torch.load(output_path))
print("Final evaluation on test set",)
parser.model.eval()
UAS, dependencies = parser.parse(test_set)
print("- test UAS: {:.2f}".format(UAS * 100.0))
print("Done!")

TESTING
Restoring the best model weights found on the dev set
Final evaluation on test set


125250it [00:00, 10883969.92it/s]      

- test UAS: 76.96
Done!


- Try to delete only the 18 pos features and check UAS

In [9]:
train_set, dev_set, test_set = load_data()
# len(train_set), len(dev_set), len(test_set)
print('2. Building parser....')
start = time.time()
parser = ablationStudyParser(train_set,pos_f=False)
print("took {:.2f} seconds".format(time.time()-start))

1. Loading data
Example 1:  {'word': ['ms.', 'haag', 'plays', 'elianti', '.'], 'pos': ['NNP', 'NNP', 'VBZ', 'NNP', '.'], 'head': [2, 3, 0, 3, 3], 'dep': ['compound', 'nsubj', 'root', 'dobj', 'punct']}
took 1.68 seconds
2. Building parser....
took 0.01 seconds


In [10]:
#before numericalize
print('Word: ',train_set[1]['word'])
# print('Pos: ',train_set[1]['pos'])
print('Head: ',train_set[1]['head'])
print('Dep: ',train_set[1]['dep'])
train_set = parser.numericalize(train_set)
dev_set = parser.numericalize(dev_set)
test_set = parser.numericalize(test_set)
#after numericalize (rootis added in front)
print('Word: ',train_set[1]['word'])
# print('Pos: ',train_set[1]['pos'])
print('Head: ',train_set[1]['head'])
print('Dep: ',train_set[1]['dep'])

Word:  ['ms.', 'haag', 'plays', 'elianti', '.']
Head:  [2, 3, 0, 3, 3]
Dep:  ['compound', 'nsubj', 'root', 'dobj', 'punct']
Word:  [5110, 258, 1318, 956, 2098, 41]
Head:  [-1, 2, 3, 0, 3, 3]
Dep:  [-1, 27, 34, 0, 17, 10]


In [11]:
print("4. Loading pretrained embeddings...",)
# config = Config()
start = time.time()
word_vectors = {}
for line in open('./data/en-cw.txt').readlines():
    we = line.strip().split() #we = word embeddings - first column: word;  the rest is embedding
    word_vectors[we[0]] = [float(x) for x in we[1:]] #{word: [list of 50 numbers], nextword: [another list], so on...}
    
#create an empty embedding matrix holding the embedding lookup table (vocab size, embed dim)
#we use random.normal instead of zeros, to keep the embedding matrix arbitrary in case word vectors don't exist....
embeddings_matrix = np.asarray(np.random.normal(0, 0.9, (parser.n_tokens, 50)), dtype='float32')

for token in parser.tok2id:
        i = parser.tok2id[token]
        if token in word_vectors:
            embeddings_matrix[i] = word_vectors[token]
        elif token.lower() in word_vectors:
            embeddings_matrix[i] = word_vectors[token.lower()]
print("Embedding matrix shape (vocab, emb size): ", embeddings_matrix.shape)
print("took {:.2f} seconds".format(time.time() - start))

print("5. Preprocessing training data...",)
start = time.time()
train_examples_without12dep = parser.create_instances(train_set)
print("took {:.2f} seconds".format(time.time() - start))

4. Loading pretrained embeddings...
Embedding matrix shape (vocab, emb size):  (5111, 50)
took 1.96 seconds
5. Preprocessing training data...
took 0.96 seconds


In [17]:
#create directory if it does not exist for saving the weights...
output_path = "./output/model_withoutpos.weights"

print(80 * "=")
print("TRAINING")
print(80 * "=")
    
model = ParserModel(embeddings_matrix, n_features = 30)
parser.model = model

start = time.time()
train(parser, train_examples_without12dep, dev_set, output_path,
      batch_size=1024, n_epochs=10, lr=0.0005, dep_f = True)

TRAINING
Epoch 1 out of 10


100%|██████████| 48/48 [00:01<00:00, 25.74it/s]


Average Train Loss: 0.5993020428965489
Evaluating on dev set


125250it [00:00, 41789561.37it/s]      


- dev UAS: 48.94
New best dev UAS! Saving model.

Epoch 2 out of 10


...

- dev UAS: 65.06

Epoch 10 out of 10


100%|██████████| 48/48 [00:01<00:00, 26.50it/s]


Average Train Loss: 0.1446195289803048
Evaluating on dev set


125250it [00:00, 35505310.62it/s]      

- dev UAS: 66.06



In [18]:
print(80 * "=")
print("TESTING")
print(80 * "=")

print("Restoring the best model weights found on the dev set")
parser.model.load_state_dict(torch.load(output_path))
print("Final evaluation on test set",)
parser.model.eval()
UAS, dependencies = parser.parse(test_set)
print("- test UAS: {:.2f}".format(UAS * 100.0))
print("Done!")

TESTING
Restoring the best model weights found on the dev set
Final evaluation on test set


125250it [00:00, 27777949.24it/s]      

- test UAS: 67.75
Done!


| Ablation study   | dev UAS | test UAS |
|------------------|---------|----------|
| word + pos + dep | 73.32 % | 74.01 %  |
| word + pos       | 75.53 % | 76.96 %  |
| word + dep       | 66.07 % | 67.75 %  |

In ablation study, It found that existing POS embedding impact to dev UAS accuracy which is same as paper mention that "POS tags of two tokens already capture most of the label information between them". 

I assume that POS represent of the grammatical roles played by words in a sentence but dep(arc label) provid only relationship between words in a sentence.

# 3. Comparison 
Do another comparison study testing the embedding
Chaky uses some embedding
Try to use (1) glove embedding (smallest), (2) nn.Embedding (train from scratch) and compare with Chaky's embedding

## 3.1 glove embedding (smallest)

In [37]:
from gensim.test.utils import datapath
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec

#you have to put this file in some python/gensim directory; just run it and it will inform where to put....
glove_file = datapath('glove.6B.50d.txt')
model_gesim = KeyedVectors.load_word2vec_format(glove_file, binary=False, no_header=True)

In [36]:
train_set, dev_set, test_set = load_data()
# len(train_set), len(dev_set), len(test_set)
print('2. Building parser....')
start = time.time()
parser = Parser(train_set)
print("took {:.2f} seconds".format(time.time()-start))

train_set = parser.numericalize(train_set)
dev_set = parser.numericalize(dev_set)
test_set = parser.numericalize(test_set)

1. Loading data
Example 1:  {'word': ['ms.', 'haag', 'plays', 'elianti', '.'], 'pos': ['NNP', 'NNP', 'VBZ', 'NNP', '.'], 'head': [2, 3, 0, 3, 3], 'dep': ['compound', 'nsubj', 'root', 'dobj', 'punct']}
took 1.73 seconds
2. Building parser....
took 0.02 seconds


In [39]:
print("4. Loading pretrained embeddings...",)
# config = Config()
start = time.time()
# word_vectors = {}
# for line in open('./data/en-cw.txt').readlines():
#     we = line.strip().split() #we = word embeddings - first column: word;  the rest is embedding
#     word_vectors[we[0]] = [float(x) for x in we[1:]] #{word: [list of 50 numbers], nextword: [another list], so on...}
    
#create an empty embedding matrix holding the embedding lookup table (vocab size, embed dim)
#we use random.normal instead of zeros, to keep the embedding matrix arbitrary in case word vectors don't exist....
embeddings_matrix = np.asarray(np.random.normal(0, 0.9, (parser.n_tokens, 50)), dtype='float32')

for token in parser.tok2id:
        i = parser.tok2id[token]
        if token in model_gesim: #Changeing word_vector from en-cw.txt to glove.6B.50d.txt instead
            embeddings_matrix[i] = model_gesim[token]
        elif token.lower() in model_gesim:
            embeddings_matrix[i] = model_gesim[token.lower()]
print("Embedding matrix shape (vocab, emb size): ", embeddings_matrix.shape)
print("took {:.2f} seconds".format(time.time() - start))

print("5. Preprocessing training data...",)
start = time.time()
train_examples = parser.create_instances(train_set)
print("took {:.2f} seconds".format(time.time() - start))

4. Loading pretrained embeddings...
Embedding matrix shape (vocab, emb size):  (5157, 50)
took 0.02 seconds
5. Preprocessing training data...
took 0.95 seconds


In [50]:
#create directory if it does not exist for saving the weights...
output_path = "./output/model_gesim.weights"

print(80 * "=")
print("TRAINING")
print(80 * "=")
    
model = ParserModel(embeddings_matrix)
parser.model = model

start = time.time()
train(parser, train_examples, dev_set, output_path,
      batch_size=1024, n_epochs=10, lr=0.0005)

TRAINING
Epoch 1 out of 10


100%|██████████| 48/48 [00:02<00:00, 18.59it/s]


Average Train Loss: 0.6329304867734512
Evaluating on dev set


125250it [00:00, 10427275.68it/s]      


- dev UAS: 54.52
New best dev UAS! Saving model.

Epoch 2 out of 10


...

- dev UAS: 70.46
New best dev UAS! Saving model.

Epoch 10 out of 10


100%|██████████| 48/48 [00:02<00:00, 19.00it/s]


Average Train Loss: 0.14209641112635532
Evaluating on dev set


125250it [00:00, 10889384.49it/s]      

- dev UAS: 71.90
New best dev UAS! Saving model.



In [51]:
print(80 * "=")
print("TESTING")
print(80 * "=")

print("Restoring the best model weights found on the dev set")
parser.model.load_state_dict(torch.load(output_path))
print("Final evaluation on test set",)
parser.model.eval()
UAS, dependencies = parser.parse(test_set)
print("- test UAS: {:.2f}".format(UAS * 100.0))
print("Done!")

TESTING
Restoring the best model weights found on the dev set
Final evaluation on test set


125250it [00:00, 10867084.03it/s]      

- test UAS: 73.94
Done!


## 3.2 nn.Embedding (train from scratch)

In [52]:
train_set, dev_set, test_set = load_data()

1. Loading data
Example 1:  {'word': ['ms.', 'haag', 'plays', 'elianti', '.'], 'pos': ['NNP', 'NNP', 'VBZ', 'NNP', '.'], 'head': [2, 3, 0, 3, 3], 'dep': ['compound', 'nsubj', 'root', 'dobj', 'punct']}
took 1.58 seconds


In [72]:
# corpus_tokenized_dev = [word['word'] for word in dev_set]
# corpus_tokenized_test = [word['word'] for word in test_set]

In [3]:
#1. tokenize
corpus_tokenized_train = [word['word'] for word in train_set]
corpus_tokenized_train[1]

['ms.', 'haag', 'plays', 'elianti', '.']

In [4]:
#2. numericalize (vocab)
#2.1 get all the unique words
#we want to flatten unit (basically merge all list)
flatten = lambda l: [item for sublist in l for item in sublist]
vocabs = list(set(flatten(corpus_tokenized_train)))

#2.2 assign id to all these vocabs
word2index = {v: idx for idx, v in enumerate(vocabs)}

#adding unknown word
vocabs.append('<UNK>')
word2index['<UNK>'] = len(vocabs) - 1

voc_size = len(vocabs)
voc_size

5070

In [5]:
#use Counter to first count stuffs
from collections import Counter
# print(corpus_tokenized)

#count the frequency of each word
#we somehow need this to claiclate the probability Pi
X_i = Counter(flatten(corpus_tokenized_train)) #merge all list ... (flattten is a function I defines)
# X_i['apple'] #get the probability of apple

In [6]:
skip_grams = []
cbows = []
window_size = 2
#for each corpus
for sent in corpus_tokenized_train:
    for i in range(1,len(sent)-window_size): #start from 2 to second last
        context_word = []
        # print(sent[i])
        center_word = sent[i]
        for j in range(window_size):
            outside_word = [sent[i-j-1],sent[i+j+1]] #window_size adjustable
            #here we want to create (banana, apple), (banana, fruit) append to some list
            for o in outside_word:
                context_word.append(o)
                skip_grams.append((center_word,o))
            cbows.append((context_word,center_word))

In [7]:
#since we have these occcurences, we can count, to make our co-occurence matrix!!!
X_ik_skipgram = Counter(skip_grams)
# X_ik_skipgram

In [8]:
def weighting(w_i,w_j,X_ik): #we need w_i and w_j, because we can try its-co-occurrences, if it's too big, we scale it down
    #check whether the co-occurrences between these two word exits??
    try:
        x_ij = X_ik[(w_i,w_j)]
    except:
        x_ij = 1 #why one, so that the probability thingly won't break....
    
    #maximum co-occurrences; we follow the paper
    x_max = 100
    alpha = 0.75

    #if the co-occurrences does not exceed x_max, cale it down based on some alpha
    if x_ij < x_max:
        result = (x_ij/x_max)**alpha
    else:
        result = 1 #this is the maximum probability you can havve
    
    return result

In [9]:
#now apply this weighting to all possible pairs
from itertools import combinations_with_replacement

X_ik = {} #for keeping the co-occurrences
weighting_dic = {} #for keeping all the probability after passing through the weighting function

for bigram in combinations_with_replacement(vocabs,2): #we need to also think its reverse
    # print(bigram)
    #if this bigram exists in X_ik_skipgrams
    #we gonna add this to our c-occurence matrix
    if X_ik_skipgram.get(bigram) is not None:
        cooc = X_ik_skipgram[bigram] #get the co-occurrences
        X_ik[bigram] = cooc + 1 #this is agian basically label smoothing.... (stability issue (especailly when divide something))
        X_ik[(bigram[1],bigram[0])] = cooc + 1 #trick to get all pairs
    else: #otherwise, do nothing
        pass
    #apply the weighting function using this co-occurrence matrix thingly
    weighting_dic[bigram] = weighting(bigram[0],bigram[1],X_ik)
    weighting_dic[(bigram[1],bigram[0])] = weighting(bigram[1],bigram[0],X_ik)

In [10]:
import math

def random_batch(batch_size, word_sequence, skip_grams, X_ik, weighting_dic):
    #loop through skipgram, and change it id because when sending model, it must number 
    skipg_grams_id = [(word2index[skip_gram[0]],word2index[skip_gram[1]]) for skip_gram in skip_grams]
    #randomly pick 'batch_size' indexes
    number_of_choices = len(skipg_grams_id)
    random_index = np.random.choice(number_of_choices, batch_size, replace=False) #no repeating indexes among  these random indexes
    
    # print(random_index)
    random_inputs = [] #xi, wi (in batches)
    random_labels = [] #xj, wj (in batches)
    random_coocs  = [] #xij (in batches)
    random_weighting = [] #f(xij) (in batches)
    #for each of the sample in these indexes
    for i in random_index:
        random_inputs.append([skipg_grams_id[i][0]]) #same reason why I put bracket here....
        random_labels.append([skipg_grams_id[i][1]])

        #get coocs
        #first check whether it exists.....
        pair = skip_grams[i] #e.g., ['banana','fruit']
        try:
            cooc = X_ik[pair]
        except:
            cooc = 1 #label smoothing

        random_coocs.append([math.log(cooc)]) #1. why log, #2 why bracket -> size ==> (,1) #my neural network excepts (,1)
        
        #for weighting
        weighting = weighting_dic[pair] #why not user try... maybe it does not exist
        random_weighting.append(weighting)

    return np.array(random_inputs),np.array(random_labels),np.array(random_coocs),np.array(random_weighting)
        #return xi,xj
        #return cooc Xij
        #return weighting

In [57]:
class GloVe(nn.Module):
    def __init__(self,voc_size, emb_size):
        super(GloVe,self).__init__()
        self.embedding_center_word = nn.Embedding(voc_size, emb_size) #is a lookup table mapping all ids in voc_size, into some vector of size emb_size
        self.embedding_outside_word = nn.Embedding(voc_size, emb_size)
        
        self.bias_i = nn.Embedding(voc_size, 1)
        self.bias_j = nn.Embedding(voc_size, 1)
        
    def forward(self, center_words, outside_words, coocs, weighting):
        #get the embedding of center_words and outside_words
        center_embeds = self.embedding_center_word(center_words)
        outside_embeds = self.embedding_outside_word(outside_words)

        #create biases #create unique embedding (voc_size,1)
        inner_product = center_embeds.bmm(center_embeds.transpose(1, 2)).squeeze(2)
        bias_i = self.bias_i(center_words).squeeze(1) #center
        bias_j = self.bias_j(outside_words).squeeze(1) #target
        #do the product between wi and wj
        loss = weighting * torch.pow(inner_product + bias_i + bias_j - coocs, 2)
        
        return torch.sum(loss)

In [58]:
voc_size = len(vocabs)
batch_size = 2 #why? no reason
emb_size = 2 #why? no reason; usually 50,100, 300 but 2 so we can plot (50 can also plot, but need PCA)
model = GloVe(voc_size,emb_size)

criterion = nn.CrossEntropyLoss() #-log
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [15]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [16]:
import time
num_epochs = 5000
#for epoch
start_time = time.time()
for epoch in range(num_epochs):
    #get random batch
    input, label, cooc, weightin = random_batch(batch_size,corpus_tokenized_train,skip_grams,X_ik,weighting_dic)
    input_batch = torch.LongTensor(input)
    label_batch = torch.LongTensor(label)
    cooc_batch = torch.FloatTensor(cooc)
    weightin_batch = torch.FloatTensor(weightin)
    
    # print(input_batch.shape,label_batch.shape,cooc_batch.shape,weightin_batch.shape)

    #loss = model
    loss = model(input_batch,label_batch,cooc_batch,weightin_batch)
    #backpropagate
    loss.backward()

    #update alpha
    optimizer.step()

    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    #print epoch loss
    if (epoch + 1) % 1000 == 0:
        print(f'Epoch {epoch+1} | Loss {loss:.6f} | Time : {epoch_mins}m {epoch_secs}s')

torch.save(model.state_dict(), "./model/dep_GloVE.pkl.pth")
with open("./model/"+"dep_GloVE.pkl",'wb') as f:
    pickle.dump(model,f)
    # break

Epoch 1000 | Loss 1.754441 | Time : 0m 27s
Epoch 2000 | Loss 3.536602 | Time : 0m 49s
Epoch 3000 | Loss 2.018674 | Time : 1m 12s
Epoch 4000 | Loss 1.164082 | Time : 1m 34s
Epoch 5000 | Loss 1.975587 | Time : 1m 56s


In [53]:
train_set, dev_set, test_set = load_data()
# len(train_set), len(dev_set), len(test_set)
print('2. Building parser....')
start = time.time()
parser = Parser(train_set)
print("took {:.2f} seconds".format(time.time()-start))

train_set = parser.numericalize(train_set)
dev_set = parser.numericalize(dev_set)
test_set = parser.numericalize(test_set)

1. Loading data
Example 1:  {'word': ['ms.', 'haag', 'plays', 'elianti', '.'], 'pos': ['NNP', 'NNP', 'VBZ', 'NNP', '.'], 'head': [2, 3, 0, 3, 3], 'dep': ['compound', 'nsubj', 'root', 'dobj', 'punct']}
took 1.91 seconds
2. Building parser....
took 0.02 seconds


In [60]:
#Load Model From GloVE Scratch
model_scratch = model.load_state_dict(torch.load("./model/dep_GloVE.pkl.pth"))

In [61]:
print("4. Loading pretrained embeddings...",)
# config = Config()
start = time.time()
# word_vectors = {}
# for line in open('./data/en-cw.txt').readlines():
#     we = line.strip().split() #we = word embeddings - first column: word;  the rest is embedding
#     word_vectors[we[0]] = [float(x) for x in we[1:]] #{word: [list of 50 numbers], nextword: [another list], so on...}
    
#create an empty embedding matrix holding the embedding lookup table (vocab size, embed dim)
#we use random.normal instead of zeros, to keep the embedding matrix arbitrary in case word vectors don't exist....
embeddings_matrix = np.asarray(np.random.normal(0, 0.9, (parser.n_tokens, 50)), dtype='float32')

for token in parser.tok2id:
        i = parser.tok2id[token]
        if token in model_scratch: #Changeing word_vector from en-cw.txt to model_scratch instead
            embeddings_matrix[i] = model_scratch[token]
        elif token.lower() in model_scratch:
            embeddings_matrix[i] = model_scratch[token.lower()]
print("Embedding matrix shape (vocab, emb size): ", embeddings_matrix.shape)
print("took {:.2f} seconds".format(time.time() - start))

print("5. Preprocessing training data...",)
start = time.time()
train_examples = parser.create_instances(train_set)
print("took {:.2f} seconds".format(time.time() - start))

4. Loading pretrained embeddings...
Embedding matrix shape (vocab, emb size):  (5157, 50)
took 0.01 seconds
5. Preprocessing training data...
took 0.74 seconds


In [62]:
#create directory if it does not exist for saving the weights...
output_path = "./output/model_scratch.weights"

print(80 * "=")
print("TRAINING")
print(80 * "=")
    
model = ParserModel(embeddings_matrix)
parser.model = model

start = time.time()
train(parser, train_examples, dev_set, output_path,
      batch_size=1024, n_epochs=10, lr=0.0005)

TRAINING
Epoch 1 out of 10


100%|██████████| 48/48 [00:02<00:00, 18.51it/s]


Average Train Loss: 0.6345977007100979
Evaluating on dev set


125250it [00:00, 14717783.83it/s]      


- dev UAS: 50.59
New best dev UAS! Saving model.

Epoch 2 out of 10


...

- dev UAS: 70.55
New best dev UAS! Saving model.

Epoch 10 out of 10


100%|██████████| 48/48 [00:02<00:00, 18.93it/s]


Average Train Loss: 0.150922700141867
Evaluating on dev set


125250it [00:00, 10887353.39it/s]      

- dev UAS: 71.45
New best dev UAS! Saving model.



In [63]:
print(80 * "=")
print("TESTING")
print(80 * "=")

print("Restoring the best model weights found on the dev set")
parser.model.load_state_dict(torch.load(output_path))
print("Final evaluation on test set",)
parser.model.eval()
UAS, dependencies = parser.parse(test_set)
print("- test UAS: {:.2f}".format(UAS * 100.0))
print("Done!")

TESTING
Restoring the best model weights found on the dev set
Final evaluation on test set


125250it [00:00, 10011750.57it/s]      

- test UAS: 72.86
Done!


| Ablation study    | dev UAS | test UAS |
|-------------------|---------|----------|
| en-cw.txt         | 73.32 % | 74.01 %  |
| glove.6B.50d      | 71.90 % | 73.94 %  |
| embedding scratch | 71.45 % | 72.86 %  |

# 4. Testing 
Do some testing, compare 2-3 sentences with spaCy and see whether our neural network gives the same dependency.

In [20]:
#loading test_set
train_set, dev_set, test_set = load_data()

1. Loading data
Example 1:  {'word': ['ms.', 'haag', 'plays', 'elianti', '.'], 'pos': ['NNP', 'NNP', 'VBZ', 'NNP', '.'], 'head': [2, 3, 0, 3, 3], 'dep': ['compound', 'nsubj', 'root', 'dobj', 'punct']}
took 1.83 seconds


In [21]:
test_sentence = [sent for sent in test_set if len(sent['word']) < 10]  #selec only short sentence
len(test_sentence) 

37

In [24]:
test_list = [sent['word'] for sent in test_sentence] #select only word sentences
doc = [' '.join(map(str,t)) for t in test_list] #join as the sentence
len(doc)

37

In [26]:
import spacy
def Spacy_Parser(doc):
    nlp = spacy.load("en_core_web_sm")
    sum_test_set_spacy = list()
    for each_doc in doc:
        d = nlp(each_doc)
        test_set_spacy = dict()
        test_set_spacy['word'],test_set_spacy['pos'],test_set_spacy['head'],test_set_spacy['dep'] = list(),list(),list(),list()
        for token in d:
            test_set_spacy['word'].append(token.text)
            test_set_spacy['pos'].append(token.tag_)
            test_set_spacy['head'].append(each_doc.index(token.head.text))
            test_set_spacy['dep'].append(token.dep_)
        sum_test_set_spacy.append(test_set_spacy)
    return sum_test_set_spacy

In [27]:
#comparing pos head dep
test_set_spacy = Spacy_Parser(doc)

In [28]:
for i in range(len(test_sentence)):
    print(f'sentence {i+1}')
    print('Word        :', test_sentence[i]['word'])
    print('Dep default :', test_sentence[i]['dep'])
    print('Dep Spacy   :', test_set_spacy[i]['dep'])
    print(80 * "=")
    if i == 2:
        break

sentence 1
Word        : ['no', ',', 'it', 'was', "n't", 'black', 'monday', '.']
Dep default : ['discourse', 'punct', 'nsubj', 'cop', 'neg', 'compound', 'root', 'punct']
Dep Spacy   : ['intj', 'punct', 'nsubj', 'ROOT', 'neg', 'amod', 'npadvmod', 'punct']
sentence 2
Word        : ['the', 'finger-pointing', 'has', 'already', 'begun', '.']
Dep default : ['det', 'nsubj', 'aux', 'advmod', 'root', 'punct']
Dep Spacy   : ['det', 'compound', 'punct', 'nsubj', 'aux', 'advmod', 'ROOT', 'punct']
sentence 3
Word        : ['``', 'the', 'equity', 'market', 'was', 'illiquid', '.']
Dep default : ['punct', 'det', 'compound', 'nsubj', 'cop', 'root', 'punct']
Dep Spacy   : ['punct', 'punct', 'det', 'compound', 'nsubj', 'ROOT', 'attr', 'punct']


Depenedncy between CoNLL-2003 and Spacy has shown almost similarity; however, some punctuation Spacy doesn't count as one such as quotation mark in sentence 3, Spacy seperate punctuation as 2 items 